# A First NN with TensorFlow

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [2]:
def get_weights(n_features, n_labels):
    """
    Return TensorFlow weights
    :param n_features: Number of features
    :param n_labels: Number of labels
    :return: TensorFlow weights
    """
    # TODO: Return weights
    return tf.Variable(tf.random.truncated_normal((n_features, n_labels)), name="W", trainable=True)


def get_biases(n_labels):
    """
    Return TensorFlow bias
    :param n_labels: Number of labels
    :return: TensorFlow bias
    """
    # TODO: Return biases
    return tf.Variable(tf.zeros(n_labels), name="b", trainable=True)


def linear(x, w, b):
    """
    Return linear function in TensorFlow
    :param x: TensorFlow input
    :param w: TensorFlow weights
    :param b: TensorFlow biases
    :return: TensorFlow linear function
    """
    # TODO: Linear Function (xW + b)
    return tf.add(tf.matmul(x, w), b)

In [5]:
def mnist_features_labels(n_images):
    """
    Gets the first <n> images from the MNIST dataset
    """
    mnist_features = []
    mnist_labels = []

    (train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data(
        "/files/cedric/datasets/udacity/ai_nanodegree/keras/mnist.npz")
    print("Total number of training images: {}".format(train_images.shape[0]))
    print("Total number of testing images: {}".format(train_images.shape[1]))
    return train_images[:n_images], train_labels[:n_images]

In [10]:
# Number of features (28*28 image is 784 features)
n_features = 784
# Number of labels
n_labels = 10

# Training data
train_features, train_labels = mnist_features_labels(60000)
train_features = train_features.astype(np.float32).reshape((60000, -1)) / 255. # reshape and normalize
print(f"Training features: {train_features.shape}, {train_features.dtype}")
train_labels = pd.get_dummies(train_labels).values.astype(np.float32)
print(f"Training labels: {train_labels.shape}, {train_labels.dtype}")

# Features and Labels
features = tf.convert_to_tensor(train_features, name="x")
labels = tf.convert_to_tensor(train_labels, name="y")

# Model
w = get_weights(n_features, n_labels)
b = get_biases(n_labels)

# Loss function
cce = tf.keras.losses.CategoricalCrossentropy()

# Optimizer
learning_rate = 0.08
optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)

Total number of training images: 60000
Total number of testing images: 28
Training features: (60000, 784), float32
Training labels: (60000, 10), float32


In [11]:
@tf.function
def session(features, labels, w, b, cce, optimizer):
    with tf.GradientTape() as tape:
        # Records the gradient during the forward pass
        # Linear Function xW + b
        logits = linear(features, w, b)   
        # Softmax
        prediction = tf.nn.softmax(logits)
        # Loss
        loss = cce(prediction, labels)
    # Get the gradients for weights and bias
    gradients = tape.gradient(loss, [w, b])
    
    # Apply optimizer (one step)
    optimizer.apply_gradients(zip(gradients, [w, b]))
    
    # End
    return [w, b, logits, prediction, loss, gradients]

outputs = session(features, labels, w, b, cce, optimizer)

print('Weights: {}'.format(outputs[0]))
print('Bias: {}'.format(outputs[1]))
print('Scores: {}'.format(outputs[2]))
print('Softmax: {}'.format(outputs[3]))
print('Loss: {}'.format(outputs[4]))
print('Gradients [w]: {}'.format(outputs[5][0]))
print('Gradients [b]: {}'.format(outputs[5][1]))

Weights: [[ 0.54152197 -1.1262169   0.29850656 ...  0.22147019  1.069837
   0.45944366]
 [ 0.18941663 -0.04188945 -0.831536   ... -1.2713394  -0.99510354
  -0.15013368]
 [-1.3394647   0.07296699 -0.14267394 ...  0.3440815   1.6246268
  -1.1913354 ]
 ...
 [ 0.9428878   0.29960948  0.77241534 ...  0.02454096  1.0591465
  -0.9935228 ]
 [ 1.8036964  -0.33023098  0.10627091 ... -0.31757647  0.35502246
   0.38793278]
 [-0.6429783  -0.34071702 -1.8948934  ...  0.73784846 -0.17394464
  -0.92118126]]
Bias: [-0.00010709  0.0076489  -0.00035305 -0.00205179  0.00264403  0.00017037
  0.00038194 -0.00109777 -0.00068567 -0.00654988]
Scores: [[  1.0122232   11.665441    -5.674629   ...  14.019657    -8.155622
    8.027456  ]
 [  2.6679661    1.4408989   -2.8300245  ...   3.3340604   -9.67888
    4.3264217 ]
 [ 15.554178     2.21923     -9.6022835  ...  -4.3050623    2.2486703
    6.012452  ]
 ...
 [ -2.747803    26.859623     6.7662134  ...   5.731075    -0.44590807
    1.8070089 ]
 [ 11.067545     2.